# Data conversion to meshes and "graphs"
(here: graph is centers and vectors) for 3D visualization

In [88]:
# napari-spatialdata dev Python 3.12.8
import pandas as pd
import napari
import numpy as np
from tqdm import tqdm
import vedo
from pathlib import Path
data_dir = Path("/Users/edmz/data/st/Fang_preprint/mouse_cortex_100um/")
path = data_dir / "feature_boundaries.pkl"
# 76355 instances
# 305420 vectors

In [ ]:
# 17s
df = pd.read_pickle(path)

In [3]:
# 2 s
z_column = 'center_z'
df = df.groupby('id')[z_column].apply(
    pd.Series.to_list).to_frame().join(
        df.groupby('id')["geometry"].apply(pd.Series.to_list).to_frame())
df["instance_id"] = range(len(df))


In [ ]:
# 7.5'
n = -1
tolerance = 2
points_dict = {}
centers_dict = {}
meshes_dict = {}
mins_dict = {}
maxs_dict = {}
for i, row in tqdm(df.head(n).iterrows(), total=len(df.head(n))):
    points = [xy + (z,) for polygon, z in zip(row["geometry"], row[z_column]) for xy in tuple(polygon.exterior.simplify(tolerance=tolerance).coords) ]
    points_dict[i] = np.array(points)
    centers_dict[i] = points_dict[i].mean(axis=0)
    min_x, min_y, min_z = np.min(points_dict[i], axis=0)
    mins_dict[i] = (min_x, min_y, min_z)
    max_x, max_y, max_z = np.max(points_dict[i], axis=0)
    maxs_dict[i] = (max_x, max_y, max_z)
    point_cloud = vedo.Points(points_dict[i])
    meshes_dict[i] = vedo.ConvexHull(point_cloud).triangulate().clean()
df["mesh"] = df.index.map(meshes_dict)

  0%|          | 0/76355 [00:00<?, ?it/s]

 25%|██▍       | 18997/76355 [01:50<05:17, 180.81it/s]2025-03-17 06:32:11.547 ( 204.857s) [          3B18FC]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x31dc80c70): 1 degenerate triangles encountered, mesh quality suspect
2025-03-17 06:32:11.599 ( 204.909s) [          3B18FC]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x31dd63b40): 1 degenerate triangles encountered, mesh quality suspect
100%|██████████| 76355/76355 [07:23<00:00, 171.98it/s]


In [ ]:
# Maybe not required
df.dropna(subset=["mesh"], inplace=True)

In [ ]:
df["mesh"] = pd.Series(meshes_dict)
df["center"] = pd.Series(centers_dict)
df["n_faces"] = df.mesh.apply(lambda x: len(x.cells))
df["n_vertices"] = df.mesh.apply(lambda x: len(x.vertices))
df[["min_x", "min_y", "min_z"]] = pd.DataFrame(mins_dict).T
df[["max_x", "max_y", "max_z"]] = pd.DataFrame(maxs_dict).T

# Save meshes
- TypeError: cannot pickle 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLProperty' object
- Save features separately

In [103]:
df[df.difference("mesh")].to_pickle(data_dir / "boundaries_features.pkl")

AttributeError: 'DataFrame' object has no attribute 'difference'

In [102]:
# save meshes to vtk
# But very long to store into ply and obj
meshes_dir = (data_dir / "meshes")
meshes_dir.mkdir(exist_ok=True)
for extension in ["vtk", "ply", "obj", "stl"]:
    for i, row in tqdm(df.iterrows(), total=len(df)):
        row["mesh"].write(str(meshes_dir / f"{i}.{extension}"))

100%|██████████| 76355/76355 [00:28<00:00, 2685.37it/s]


# Concatenate meshes and ids

In [16]:
# concatenate vedo meshes
collection = vedo.merge(df.mesh.tolist())

In [ ]:
for extension in tqdm(["vtk", "obj", "stl"]):
    collection.write(str(data_dir / f"meshes.{extension}"))
    break

In [19]:
face_seg_id = []
for i in range(len(df)):
    face_seg_id.extend([i]*df.iloc[i]["n_faces"])
vertex_seg_id = []
for i in range(len(df)):
    vertex_seg_id.extend([i]*df.iloc[i]["n_vertices"])

# Compute vectors

In [85]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 3
kn = NearestNeighbors(n_neighbors=n_neighbors)
kn.fit(np.stack(df["center"].values))
_, indices = kn.kneighbors(np.stack(df["center"].values))

pairs = []
for row in indices:
    for neighbor in row[1:]:
        pairs.append([row[0], neighbor])
pairs = np.array(pairs)

pairs = np.unique(pairs[np.argsort(pairs, axis=0)[:,0]], axis=0)

centers = np.array(list(centers_dict.values()))

vectors = centers[pairs]
vectors[:,1] = centers[pairs[:,1]] - centers[pairs[:,0]]

# Napari visualization

In [ ]:
v = napari.Viewer()
v.add_surface((collection.vertices, np.array(collection.cells), np.array(vertex_seg_id)), name="mesh", opacity=0.5, colormap="viridis")

<Points layer 'centers' at 0x6c9b26840>

2025-03-17 06:49:13.750 python[45887:3873020] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-17 06:49:13.750 python[45887:3873020] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [46]:
v.add_points(np.stack(df["center"].values), size=4, face_color='green', border_color="#00000000", name="centers")

<Points layer 'centers' at 0x45ed79d90>

In [86]:
v.add_vectors(vectors)

<Vectors layer 'vectors' at 0x3e1419400>

2025-03-17 07:27:50.954 python[45887:3873020] _TIPropertyValueIsValid called with 16 on nil context!
2025-03-17 07:27:50.954 python[45887:3873020] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-03-17 07:27:50.954 python[45887:3873020] Text input context does not respond to _valueForTIProperty:
